In [1]:
%matplotlib inline

In [2]:
cd ..

C:\Projects\python\recommender


In [3]:
from functools import partial
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [4]:
from datasets import SeqStackoverflow
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss

In [5]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
NEG_SAMPLE = 5
item_path = Path("./inputs/stackoverflow/item.csv")

In [6]:
db = SeqStackoverflow(data_path=item_path, min_user=4)
db

Original user size: 75572
Filtered user size: 14795
Original item dataframe shape: (299845, 9)
Filtered item dataframe shape: (217223, 9)
Remove number of questions less than 10 date
Filter item dateframe shape: (217146, 9)


In [7]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE,
             neg_sample=NEG_SAMPLE)

## Create Criterion

In [8]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [9]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x000001B49EE2A1E0>, 1, 1)

In [10]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x000001B49EE83400>, 1, 1, 1)

## Train model

### Hyper-parameter

In [11]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

## Train fm model

In [12]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [14]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [15]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [16]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [18]:
fm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [19]:
fm_learner.fit(epoch=5,
               log_dir=get_log_dir(ds_type='seq_stackoverflow',
                                   model_type='fm'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 156350.40846478133
Epoch 0 step 1: training accuarcy: 0.3155
Epoch 0 step 1: training loss: 151627.33504858427
Epoch 0 step 2: training accuarcy: 0.3795
Epoch 0 step 2: training loss: 147201.86598852006
Epoch 0 step 3: training accuarcy: 0.41100000000000003
Epoch 0 step 3: training loss: 142689.1626520957
Epoch 0 step 4: training accuarcy: 0.481
Epoch 0 step 4: training loss: 138534.1221418247
Epoch 0 step 5: training accuarcy: 0.494
Epoch 0 step 5: training loss: 134301.95268291325
Epoch 0 step 6: training accuarcy: 0.5545
Epoch 0 step 6: training loss: 130296.83493591996
Epoch 0 step 7: training accuarcy: 0.5875
Epoch 0 step 7: training loss: 126347.68557358337
Epoch 0 step 8: training accuarcy: 0.626
Epoch 0 step 8: training loss: 122515.48753288513
Epoch 0 step 9: training accuarcy: 0.6515
Epoch 0 step 9: training loss: 118732.31667440773
Epoch 0 step 10: training accuarcy: 0.7045
Epoch 0 step 10: training loss: 115100.44612814624
Epoch 0 ste

 20%|██████████████████████████████████▌                                                                                                                                          | 1/5 [01:18<05:14, 78.70s/it]

Epoch: 1
Epoch 1 step 94: training loss: 5226.360627683903
Epoch 1 step 95: training accuarcy: 0.999
Epoch 1 step 95: training loss: 5015.117588995671
Epoch 1 step 96: training accuarcy: 1.0
Epoch 1 step 96: training loss: 4813.5409984518765
Epoch 1 step 97: training accuarcy: 1.0
Epoch 1 step 97: training loss: 4620.309040198561
Epoch 1 step 98: training accuarcy: 1.0
Epoch 1 step 98: training loss: 4433.890697158946
Epoch 1 step 99: training accuarcy: 1.0
Epoch 1 step 99: training loss: 4257.869016747632
Epoch 1 step 100: training accuarcy: 0.9995
Epoch 1 step 100: training loss: 4084.7167375647077
Epoch 1 step 101: training accuarcy: 1.0
Epoch 1 step 101: training loss: 3919.3381046011314
Epoch 1 step 102: training accuarcy: 1.0
Epoch 1 step 102: training loss: 3762.4667937167324
Epoch 1 step 103: training accuarcy: 1.0
Epoch 1 step 103: training loss: 3610.8007383504987
Epoch 1 step 104: training accuarcy: 1.0
Epoch 1 step 104: training loss: 3465.4663681259335
Epoch 1 step 105: tr

 40%|█████████████████████████████████████████████████████████████████████▏                                                                                                       | 2/5 [02:36<03:55, 78.54s/it]

...

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                     | 3/5 [03:55<02:37, 78.53s/it]

Epoch: 3
Epoch 3 step 282: training loss: 60.378811992301486
Epoch 3 step 283: training accuarcy: 1.0
Epoch 3 step 283: training loss: 60.56544144442725
Epoch 3 step 284: training accuarcy: 1.0
Epoch 3 step 284: training loss: 59.090601695054815
Epoch 3 step 285: training accuarcy: 1.0
Epoch 3 step 285: training loss: 58.7778670407271
Epoch 3 step 286: training accuarcy: 1.0
Epoch 3 step 286: training loss: 58.43989460277152
Epoch 3 step 287: training accuarcy: 1.0
Epoch 3 step 287: training loss: 57.55476142167913
Epoch 3 step 288: training accuarcy: 1.0
Epoch 3 step 288: training loss: 57.63787578484341
Epoch 3 step 289: training accuarcy: 1.0
Epoch 3 step 289: training loss: 56.80576665038936
Epoch 3 step 290: training accuarcy: 1.0
Epoch 3 step 290: training loss: 56.43283477469722
Epoch 3 step 291: training accuarcy: 1.0
Epoch 3 step 291: training loss: 55.90714193654783
Epoch 3 step 292: training accuarcy: 1.0
Epoch 3 step 292: training loss: 55.35509385390035
Epoch 3 step 293: t

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                  | 4/5 [05:13<01:18, 78.37s/it]

Epoch: 4
Epoch 4 step 376: training loss: 34.01134157192234
Epoch 4 step 377: training accuarcy: 1.0
Epoch 4 step 377: training loss: 33.324933272483094
Epoch 4 step 378: training accuarcy: 1.0
Epoch 4 step 378: training loss: 33.58446444920643
Epoch 4 step 379: training accuarcy: 1.0
Epoch 4 step 379: training loss: 33.58626178922205
Epoch 4 step 380: training accuarcy: 1.0
Epoch 4 step 380: training loss: 33.79700455288014
Epoch 4 step 381: training accuarcy: 1.0
Epoch 4 step 381: training loss: 33.12533024867399
Epoch 4 step 382: training accuarcy: 1.0
Epoch 4 step 382: training loss: 32.67779216082605
Epoch 4 step 383: training accuarcy: 1.0
Epoch 4 step 383: training loss: 33.0803382792642
Epoch 4 step 384: training accuarcy: 1.0
Epoch 4 step 384: training loss: 32.7119103369336
Epoch 4 step 385: training accuarcy: 1.0
Epoch 4 step 385: training loss: 32.23378004960491
Epoch 4 step 386: training accuarcy: 1.0
Epoch 4 step 386: training loss: 32.1579866557391
Epoch 4 step 387: trai

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:31<00:00, 78.26s/it]


In [28]:
del fm_model
T.cuda.empty_cache()

### Train HRM FM Model

In [20]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [21]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [22]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [23]:
hrm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [24]:
hrm_learner.fit(epoch=5,
                log_dir=get_log_dir('seq_stackoverflow', 'hrm'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 159449.74616477397
Epoch 0 step 1: training accuarcy: 0.4855
Epoch 0 step 1: training loss: 154508.20718114648
Epoch 0 step 2: training accuarcy: 0.51
Epoch 0 step 2: training loss: 149945.26021111847
Epoch 0 step 3: training accuarcy: 0.493
Epoch 0 step 3: training loss: 145605.51904633694
Epoch 0 step 4: training accuarcy: 0.506
Epoch 0 step 4: training loss: 141027.2265681734
Epoch 0 step 5: training accuarcy: 0.5105000000000001
Epoch 0 step 5: training loss: 136983.23592669764
Epoch 0 step 6: training accuarcy: 0.5045000000000001
Epoch 0 step 6: training loss: 133017.319562297
Epoch 0 step 7: training accuarcy: 0.495
Epoch 0 step 7: training loss: 129023.99524061031
Epoch 0 step 8: training accuarcy: 0.5075000000000001
Epoch 0 step 8: training loss: 124990.26149430107
Epoch 0 step 9: training accuarcy: 0.5005000000000001
Epoch 0 step 9: training loss: 121112.51472496541
Epoch 0 step 10: training accuarcy: 0.513
Epoch 0 step 10: training loss:

 20%|██████████████████████████████████▌                                                                                                                                          | 1/5 [01:20<05:23, 80.77s/it]

Epoch: 1
Epoch 1 step 94: training loss: 6120.881981504964
Epoch 1 step 95: training accuarcy: 0.5435
Epoch 1 step 95: training loss: 5905.930824136662
Epoch 1 step 96: training accuarcy: 0.554
Epoch 1 step 96: training loss: 5728.454368634429
Epoch 1 step 97: training accuarcy: 0.5385
Epoch 1 step 97: training loss: 5533.730901441544
Epoch 1 step 98: training accuarcy: 0.5615
Epoch 1 step 98: training loss: 5347.536542319389
Epoch 1 step 99: training accuarcy: 0.5485
Epoch 1 step 99: training loss: 5175.711708899931
Epoch 1 step 100: training accuarcy: 0.556
Epoch 1 step 100: training loss: 5011.179423012626
Epoch 1 step 101: training accuarcy: 0.556
Epoch 1 step 101: training loss: 4846.5108505765365
Epoch 1 step 102: training accuarcy: 0.5690000000000001
Epoch 1 step 102: training loss: 4732.235576822035
Epoch 1 step 103: training accuarcy: 0.5615
Epoch 1 step 103: training loss: 4586.728915962958
Epoch 1 step 104: training accuarcy: 0.5465
Epoch 1 step 104: training loss: 4434.2686

 40%|█████████████████████████████████████████████████████████████████████▏                                                                                                       | 2/5 [02:43<04:04, 81.49s/it]

...

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                     | 3/5 [04:06<02:43, 81.67s/it]

Epoch: 3
Epoch 3 step 282: training loss: 1353.9985873569228
Epoch 3 step 283: training accuarcy: 0.5825
Epoch 3 step 283: training loss: 1344.3050780022159
Epoch 3 step 284: training accuarcy: 0.596
Epoch 3 step 284: training loss: 1353.9195510319328
Epoch 3 step 285: training accuarcy: 0.597
Epoch 3 step 285: training loss: 1347.4614304166591
Epoch 3 step 286: training accuarcy: 0.6025
Epoch 3 step 286: training loss: 1338.7372483156742
Epoch 3 step 287: training accuarcy: 0.6025
Epoch 3 step 287: training loss: 1340.5008611809385
Epoch 3 step 288: training accuarcy: 0.581
Epoch 3 step 288: training loss: 1327.5749387798894
Epoch 3 step 289: training accuarcy: 0.61
Epoch 3 step 289: training loss: 1356.6349051462687
Epoch 3 step 290: training accuarcy: 0.5875
Epoch 3 step 290: training loss: 1336.8306093867122
Epoch 3 step 291: training accuarcy: 0.5955
Epoch 3 step 291: training loss: 1336.0148650790172
Epoch 3 step 292: training accuarcy: 0.609
Epoch 3 step 292: training loss: 1345

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                  | 4/5 [05:27<01:21, 81.72s/it]

Epoch: 4
Epoch 4 step 376: training loss: 1348.1471435258568
Epoch 4 step 377: training accuarcy: 0.5825
Epoch 4 step 377: training loss: 1342.0333891995263
Epoch 4 step 378: training accuarcy: 0.5875
Epoch 4 step 378: training loss: 1340.9567959929414
Epoch 4 step 379: training accuarcy: 0.59
Epoch 4 step 379: training loss: 1340.508620966616
Epoch 4 step 380: training accuarcy: 0.602
Epoch 4 step 380: training loss: 1333.850060282119
Epoch 4 step 381: training accuarcy: 0.59
Epoch 4 step 381: training loss: 1342.1477539333177
Epoch 4 step 382: training accuarcy: 0.5945
Epoch 4 step 382: training loss: 1351.6574195347023
Epoch 4 step 383: training accuarcy: 0.5905
Epoch 4 step 383: training loss: 1354.0717183966487
Epoch 4 step 384: training accuarcy: 0.582
Epoch 4 step 384: training loss: 1328.1494206470863
Epoch 4 step 385: training accuarcy: 0.602
Epoch 4 step 385: training loss: 1340.558231826908
Epoch 4 step 386: training accuarcy: 0.5915
Epoch 4 step 386: training loss: 1321.757

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:48<00:00, 81.52s/it]


In [29]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [30]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [31]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [32]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [33]:
prme_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [34]:
prme_learner.fit(epoch=5,log_dir=get_log_dir('seq_stackoverflow', 'prme'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 163625.31334404234
Epoch 0 step 1: training accuarcy: 0.493
Epoch 0 step 1: training loss: 159053.84386484543
Epoch 0 step 2: training accuarcy: 0.496
Epoch 0 step 2: training loss: 154421.37391361973
Epoch 0 step 3: training accuarcy: 0.51
Epoch 0 step 3: training loss: 150078.63620001235
Epoch 0 step 4: training accuarcy: 0.5135
Epoch 0 step 4: training loss: 144752.72417866022
Epoch 0 step 5: training accuarcy: 0.5345
Epoch 0 step 5: training loss: 141069.2796469489
Epoch 0 step 6: training accuarcy: 0.513
Epoch 0 step 6: training loss: 137534.93500742718
Epoch 0 step 7: training accuarcy: 0.498
Epoch 0 step 7: training loss: 132837.28214766507
Epoch 0 step 8: training accuarcy: 0.5165
Epoch 0 step 8: training loss: 129455.17493600384
Epoch 0 step 9: training accuarcy: 0.5005000000000001
Epoch 0 step 9: training loss: 125055.68127879125
Epoch 0 step 10: training accuarcy: 0.51
Epoch 0 step 10: training loss: 120937.32056976783
Epoch 0 step 11:

 20%|██████████████████████████████████▌                                                                                                                                          | 1/5 [01:22<05:31, 82.92s/it]

Epoch: 1
Epoch 1 step 94: training loss: 8490.246713243258
Epoch 1 step 95: training accuarcy: 0.529
Epoch 1 step 95: training loss: 8182.121943406677
Epoch 1 step 96: training accuarcy: 0.538
Epoch 1 step 96: training loss: 7983.988104257445
Epoch 1 step 97: training accuarcy: 0.531
Epoch 1 step 97: training loss: 7751.978461277806
Epoch 1 step 98: training accuarcy: 0.5295
Epoch 1 step 98: training loss: 7552.896497317663
Epoch 1 step 99: training accuarcy: 0.5305
Epoch 1 step 99: training loss: 7368.974533635669
Epoch 1 step 100: training accuarcy: 0.513
Epoch 1 step 100: training loss: 7083.361314707041
Epoch 1 step 101: training accuarcy: 0.5255
Epoch 1 step 101: training loss: 6914.690266067468
Epoch 1 step 102: training accuarcy: 0.523
Epoch 1 step 102: training loss: 6784.328299345862
Epoch 1 step 103: training accuarcy: 0.513
Epoch 1 step 103: training loss: 6534.229203209648
Epoch 1 step 104: training accuarcy: 0.5435
Epoch 1 step 104: training loss: 6336.477428611949
Epoch 1

 40%|█████████████████████████████████████████████████████████████████████▏                                                                                                       | 2/5 [02:49<04:11, 83.94s/it]

...

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                     | 3/5 [04:12<02:47, 83.66s/it]

Epoch: 3
Epoch 3 step 282: training loss: 1409.6966998501161
Epoch 3 step 283: training accuarcy: 0.587
Epoch 3 step 283: training loss: 1411.8633999444946
Epoch 3 step 284: training accuarcy: 0.5865
Epoch 3 step 284: training loss: 1381.4901869120238
Epoch 3 step 285: training accuarcy: 0.612
Epoch 3 step 285: training loss: 1388.13925670535
Epoch 3 step 286: training accuarcy: 0.5985
Epoch 3 step 286: training loss: 1382.698158967659
Epoch 3 step 287: training accuarcy: 0.602
Epoch 3 step 287: training loss: 1389.9223615245196
Epoch 3 step 288: training accuarcy: 0.603
Epoch 3 step 288: training loss: 1383.2993944610123
Epoch 3 step 289: training accuarcy: 0.6075
Epoch 3 step 289: training loss: 1398.890204059772
Epoch 3 step 290: training accuarcy: 0.5875
Epoch 3 step 290: training loss: 1390.7510149107393
Epoch 3 step 291: training accuarcy: 0.595
Epoch 3 step 291: training loss: 1387.5828021902194
Epoch 3 step 292: training accuarcy: 0.59
Epoch 3 step 292: training loss: 1398.1483

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                  | 4/5 [05:36<01:23, 83.80s/it]

Epoch: 4
Epoch 4 step 376: training loss: 1360.9900585763228
Epoch 4 step 377: training accuarcy: 0.5785
Epoch 4 step 377: training loss: 1359.267329781073
Epoch 4 step 378: training accuarcy: 0.6015
Epoch 4 step 378: training loss: 1361.7449540081498
Epoch 4 step 379: training accuarcy: 0.595
Epoch 4 step 379: training loss: 1345.396363767901
Epoch 4 step 380: training accuarcy: 0.6075
Epoch 4 step 380: training loss: 1346.4767924012322
Epoch 4 step 381: training accuarcy: 0.594
Epoch 4 step 381: training loss: 1332.4465988798675
Epoch 4 step 382: training accuarcy: 0.599
Epoch 4 step 382: training loss: 1341.1260411701455
Epoch 4 step 383: training accuarcy: 0.604
Epoch 4 step 383: training loss: 1356.9727228753327
Epoch 4 step 384: training accuarcy: 0.593
Epoch 4 step 384: training loss: 1338.5238653857832
Epoch 4 step 385: training accuarcy: 0.6055
Epoch 4 step 385: training loss: 1356.2163716840184
Epoch 4 step 386: training accuarcy: 0.5885
Epoch 4 step 386: training loss: 1354.

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [07:02<00:00, 84.56s/it]


In [41]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [13]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [14]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [15]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [16]:
trans_learner.compile(train_col='base',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=simple_loss_callback)

In [17]:
trans_learner.fit(epoch=5, log_dir=get_log_dir('seq_stackoverflow', 'trans'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 164958.5357854265
Epoch 0 step 1: training accuarcy: 0.518
Epoch 0 step 1: training loss: 160368.91942496574
Epoch 0 step 2: training accuarcy: 0.5035000000000001
Epoch 0 step 2: training loss: 155344.1250532723
Epoch 0 step 3: training accuarcy: 0.5175
Epoch 0 step 3: training loss: 151314.12961604865
Epoch 0 step 4: training accuarcy: 0.513
Epoch 0 step 4: training loss: 146592.98273384754
Epoch 0 step 5: training accuarcy: 0.496
Epoch 0 step 5: training loss: 142728.94892648255
Epoch 0 step 6: training accuarcy: 0.4975
Epoch 0 step 6: training loss: 138132.09020402856
Epoch 0 step 7: training accuarcy: 0.518
Epoch 0 step 7: training loss: 133968.60335181077
Epoch 0 step 8: training accuarcy: 0.5075000000000001
Epoch 0 step 8: training loss: 130519.33231767353
Epoch 0 step 9: training accuarcy: 0.5075000000000001
Epoch 0 step 9: training loss: 127024.94322712412
Epoch 0 step 10: training accuarcy: 0.4865
Epoch 0 step 10: training loss: 122531.0

 20%|██████████████████████████████████▌                                                                                                                                          | 1/5 [01:31<06:04, 91.09s/it]

Epoch: 1
Epoch 1 step 94: training loss: 8468.06361467602
Epoch 1 step 95: training accuarcy: 0.619
Epoch 1 step 95: training loss: 8237.7399516149
Epoch 1 step 96: training accuarcy: 0.622
Epoch 1 step 96: training loss: 8025.693425424888
Epoch 1 step 97: training accuarcy: 0.615
Epoch 1 step 97: training loss: 7775.535208786701
Epoch 1 step 98: training accuarcy: 0.629
Epoch 1 step 98: training loss: 7581.423166052558
Epoch 1 step 99: training accuarcy: 0.618
Epoch 1 step 99: training loss: 7346.008004475699
Epoch 1 step 100: training accuarcy: 0.631
Epoch 1 step 100: training loss: 7157.674886435413
Epoch 1 step 101: training accuarcy: 0.614
Epoch 1 step 101: training loss: 6975.372647907814
Epoch 1 step 102: training accuarcy: 0.64
Epoch 1 step 102: training loss: 6807.763966372768
Epoch 1 step 103: training accuarcy: 0.61
Epoch 1 step 103: training loss: 6616.774350835358
Epoch 1 step 104: training accuarcy: 0.629
Epoch 1 step 104: training loss: 6428.667614136943
Epoch 1 step 105

 40%|█████████████████████████████████████████████████████████████████████▏                                                                                                       | 2/5 [03:02<04:33, 91.18s/it]

...

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                     | 3/5 [04:33<03:02, 91.09s/it]

Epoch: 3
Epoch 3 step 282: training loss: 1321.837596538467
Epoch 3 step 283: training accuarcy: 0.7255
Epoch 3 step 283: training loss: 1325.6472751078177
Epoch 3 step 284: training accuarcy: 0.74
Epoch 3 step 284: training loss: 1327.626364497246
Epoch 3 step 285: training accuarcy: 0.733
Epoch 3 step 285: training loss: 1365.6448296152348
Epoch 3 step 286: training accuarcy: 0.7295
Epoch 3 step 286: training loss: 1354.4467639057405
Epoch 3 step 287: training accuarcy: 0.726
Epoch 3 step 287: training loss: 1329.9979958535655
Epoch 3 step 288: training accuarcy: 0.7375
Epoch 3 step 288: training loss: 1302.4285655950382
Epoch 3 step 289: training accuarcy: 0.7335
Epoch 3 step 289: training loss: 1352.3322442710369
Epoch 3 step 290: training accuarcy: 0.725
Epoch 3 step 290: training loss: 1366.7640848759622
Epoch 3 step 291: training accuarcy: 0.7205
Epoch 3 step 291: training loss: 1316.3711810879238
Epoch 3 step 292: training accuarcy: 0.7365
Epoch 3 step 292: training loss: 1341.

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                  | 4/5 [06:06<01:31, 91.69s/it]

Epoch: 4
Epoch 4 step 376: training loss: 1193.7661577366157
Epoch 4 step 377: training accuarcy: 0.7565000000000001
Epoch 4 step 377: training loss: 1213.703070783885
Epoch 4 step 378: training accuarcy: 0.763
Epoch 4 step 378: training loss: 1246.8831791309726
Epoch 4 step 379: training accuarcy: 0.7415
Epoch 4 step 379: training loss: 1194.7704351348823
Epoch 4 step 380: training accuarcy: 0.753
Epoch 4 step 380: training loss: 1239.4857555369022
Epoch 4 step 381: training accuarcy: 0.7515000000000001
Epoch 4 step 381: training loss: 1221.842278153663
Epoch 4 step 382: training accuarcy: 0.7495
Epoch 4 step 382: training loss: 1194.8138116524128
Epoch 4 step 383: training accuarcy: 0.759
Epoch 4 step 383: training loss: 1240.3861053817477
Epoch 4 step 384: training accuarcy: 0.7465
Epoch 4 step 384: training loss: 1234.7102872370465
Epoch 4 step 385: training accuarcy: 0.7515000000000001
Epoch 4 step 385: training loss: 1246.9405335407016
Epoch 4 step 386: training accuarcy: 0.755
E

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [07:39<00:00, 92.05s/it]


In [ ]:
del trans_model
T.cuda.empty_cache()